In [2]:
import os
import numpy as np
import tensorflow as tf 

os.environ['CUDA_VISIBLE_DEVICES'] = "0" #use GPU:0 only

physical_devices = tf.config.list_physical_devices('GPU')
for dev in physical_devices:  # 如果使用多块GPU时
    tf.config.experimental.set_memory_growth(dev, True)

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
#print GPU Devices
[print(x) for x in local_device_protos if x.device_type == 'GPU']
print('Tensorflow Version:',tf.__version__)

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10561793600
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5672877112755318072
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:3b:00.0, compute capability: 7.5"

Tensorflow Version: 2.4.1


# Pre-trained 2D CNN model predicts 2DUV homo and nonhomo dataset

In [3]:
# load model
model_2duv = tf.keras.models.load_model('./2DCNNDemoModel.h5')

In [4]:
model_2duv.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 159, 159, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 77, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0

## test homo data

In [10]:
# preprocessing 2duv data

# load 2duv
homo_2duv = np.load('/data1/qzhang/small_dataset_pnas/homologous/homologous_dataset.npz')['twoduv']
labels = np.load('/data1/qzhang/small_dataset_pnas/homologous/homologous_dataset.npz')['labels']

print(homo_2duv.shape)

# preprocess
# mean and std are calculated from 00-2DCNNModelDemo.ipynb 
mean = -1.59583661486646e-24
std = 1.1228893279331382e-21

homo_2duv -= mean
homo_2duv /= std

homo_2duv = homo_2duv.clip(-2,2)

# reshape
homo_2duv = homo_2duv.reshape(-1,161,161,1) # match input shape of CNN

# preprocess labels

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels_cat_2duv = encoder.fit_transform(labels)
print(encoder.classes_)
print(np.unique(labels_cat_2duv))

(1350, 25921)
['a-helix' 'b-sheet' 'other-SS']
[0 1 2]


In [11]:
# test
model_2duv.evaluate(homo_2duv,labels_cat_2duv)

43/43 [==============================] - 0s 7ms/step - loss: 0.2834 - acc: 0.9081


[0.28344300389289856, 0.9081481695175171]

In [18]:
# make confusion matrix
pre_2duv_labels = model_2duv.predict_classes(homo_2duv)

from sklearn.metrics import confusion_matrix
con_mat = confusion_matrix(labels_cat_2duv,pre_2duv_labels)
print('Confusion Matrix:\n\n',con_mat)

Confusion Matrix:

 [[449   1   0]
 [ 15 359  76]
 [  8  24 418]]


## test non-homo data

In [6]:
# load 2duv
homo_2duv = np.load('/data1/qzhang/small_dataset_pnas/nonhomologous/nonhomologous_dataset.npz')['twoduv']
labels = np.load('/data1/qzhang/small_dataset_pnas/nonhomologous/nonhomologous_dataset.npz')['labels']

print(homo_2duv.shape)

# preprocess
# mean and std are calculated from 00-2DCNNModelDemo.ipynb 
mean = -1.59583661486646e-24
std = 1.1228893279331382e-21

homo_2duv -= mean
homo_2duv /= std

homo_2duv = homo_2duv.clip(-2,2)

# reshape
homo_2duv = homo_2duv.reshape(-1,161,161,1) # match input shape of CNN

# preprocess labels

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels_cat_2duv = encoder.fit_transform(labels)
print(encoder.classes_)
print(np.unique(labels_cat_2duv))

(1350, 25921)
[0 1 2]
[0 1 2]


In [7]:
model_2duv.evaluate(homo_2duv,labels_cat_2duv)

43/43 [==============================] - 7s 9ms/step - loss: 1.7001 - acc: 0.6659


[1.700068712234497, 0.665925920009613]

In [8]:
# make confusion matrix
pre_2duv_labels = model_2duv.predict_classes(homo_2duv)

from sklearn.metrics import confusion_matrix
con_mat = confusion_matrix(labels_cat_2duv,pre_2duv_labels)
print('Confusion Matrix:\n\n',con_mat)

/home/qzhang/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Confusion Matrix:

 [[332  55  63]
 [132 149 169]
 [ 11  21 418]]
